**1. Reading In Books We Like**

In [88]:
import pandas as pd 

my_books = pd.read_csv("liked_books.csv", index_col=0)
my_books["book_id"] = my_books["book_id"].astype(str)

In [89]:
my_books

,user_id,book_id,rating,title
0,-1,17788401,3,Ugly Love
1,-1,60310757,5,"Lightlark (Lightlark, #1)"
2,-1,58410355,5,Nightbane (Lightlark #2)
3,-1,64645812,0,If Only I Had Told Her
4,-1,44676678,4,If He Had Been with Me
...,...,...,...,...
101,-1,23003206,4,Everything I Never Told You
102,-1,3437,1,The Curious Incident of the Dog in the Night-Time
103,-1,4378870,3,The Boy in the Striped Pajamas
104,-1,18690890,3,Paper Towns


In [90]:
my_books["book_id"] = my_books["book_id"].astype(str)

**2. Finding Similar Users**

In [91]:
csv_book_mapping = {}

with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

**Set that contains all of the unique books that we have read.**

In [92]:
book_set = set(my_books["book_id"])

**Every user that read the same book as us will be plaved in this overlap users dictionary.**

In [93]:
!wc -l goodreads_interactions.csv

 228648343 goodreads_interactions.csv


**Note: _ means that we do not care about that variable.**

In [94]:
overlap_users = {}

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, csv_id, _, rating, _ = line.split(",")
        
        book_id = csv_book_mapping.get(csv_id)
        
        if book_id in book_set:
            if user_id not in overlap_users:
                overlap_users[user_id] = 1
            else:
                overlap_users[user_id] += 1

In [95]:
filtered_overlap_users = set([k for k in overlap_users if overlap_users[k] > my_books.shape[0]/5])

**3. Finding Similar User Book Ratings**

In [96]:
interactions_list = []

with open("goodreads_interactions.csv") as f:
    while True:
        line = f.readline()
        if not line:
            break

        user_id, csv_id, _, rating, _ = line.strip().split(",")
        if user_id in filtered_overlap_users:
            book_id = csv_book_mapping[csv_id]
            interactions_list.append([user_id, book_id, rating]) 

**4. Creating A User /Book Matrix**

In [97]:
len(interactions_list)

1620249

In [98]:
interactions_list[0]

['281', '6216670', '5']

**DateFrame**

In [99]:
interactions = pd.DataFrame(interactions_list, columns=["user_id", "book_id", "rating"])

In [100]:
interactions = pd.concat([my_books[["user_id", "book_id", "rating"]], interactions])

In [101]:
interactions

,user_id,book_id,rating
0,-1,17788401,3
1,-1,60310757,5
2,-1,58410355,5
3,-1,64645812,0
4,-1,44676678,4
...,...,...,...
1620244,441283,35068798,0
1620245,441283,34227692,0
1620246,441283,32969999,0
1620247,441283,23705532,0


In [102]:
interactions["book_id"] = interactions["book_id"].astype(str)
interactions["user_id"] = interactions["user_id"].astype(str)
interactions["rating"] = pd.to_numeric(interactions["rating"])

In [103]:
interactions["user_id"].unique()

array(['-1', '281', '1033', '2794', '3012', '7488', '9959', '13856',
       '22124', '22864', '23326', '28128', '34134', '40239', '45510',
       '51611', '55191', '65620', '66585', '77736', '79624', '83250',
       '83293', '83470', '89997', '91247', '93625', '96010', '99975',
       '100939', '105106', '106001', '113970', '116512', '119660',
       '121119', '122431', '126987', '130849', '132126', '137133',
       '138588', '141311', '143201', '145486', '146601', '147132',
       '154116', '155631', '157964', '165733', '171930', '173042',
       '173804', '177877', '180813', '180817', '182379', '183847',
       '188693', '191515', '192434', '192632', '193311', '195488',
       '198123', '198161', '199610', '199839', '200059', '205784',
       '206213', '206369', '207960', '207969', '209647', '210169',
       '210394', '212288', '213935', '214546', '214711', '214914',
       '215878', '216545', '217460', '219243', '219898', '219965',
       '220824', '220848', '222577', '223539', '223

**Note: cat.codes means category codes.**

In [104]:
interactions["user_index"] = interactions["user_id"].astype("category").cat.codes

**User ID and Book Id - To Position.**

In [105]:
interactions["user_index"].unique()

array([  0, 103,   2,  99, 116, 187, 197,  14,  65,  70,  71, 105, 138,
       166, 182, 183, 184, 185, 186, 188, 189, 190, 191, 192, 193, 194,
       195, 196, 198,   1,   3,   4,   5,   6,   7,   8,   9,  10,  11,
        12,  13,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        66,  67,  68,  69,  72,  73,  74,  75,  76,  77,  78,  79,  80,
        81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,
        94,  95,  96,  97,  98, 100, 101, 102, 104, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 117, 118, 119, 120, 121, 122, 123,
       124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136,
       137, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150,
       151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 16

In [106]:
interactions["book_index"] = interactions["book_id"].astype("category").cat.codes

**Note 1: Sparse Matrix will be used.**

**Note 2: if there is no value in a column you just leave it blank and not take up any memory or storage space (Sparse Matrix).**

In [107]:
from scipy.sparse import coo_matrix

ratings_mat_coo = coo_matrix((interactions["rating"], (interactions["user_index"], interactions["book_index"])))

In [108]:
ratings_mat_coo

<199x461150 sparse matrix of type '<class 'numpy.int64'>'
	with 1620354 stored elements in COOrdinate format>

**Convert coo matrix to csr matrix.**

**coo matrices are a little bit easier to create which is why we initially created it in coo format and now we are going to convert it in csr format.**

In [109]:
ratings_mat = ratings_mat_coo.tocsr()

**5. Finding Users Similar To Us**

In [110]:
interactions[interactions["user_id"] == "-1"]

,user_id,book_id,rating,user_index,book_index
0,-1,17788401,3,0,116370
1,-1,60310757,5,0,387557
2,-1,58410355,5,0,383990
3,-1,64645812,0,0,398594
4,-1,44676678,4,0,366641
...,...,...,...,...,...
101,-1,23003206,4,0,200857
102,-1,3437,1,0,341345
103,-1,4378870,3,0,365383
104,-1,18690890,3,0,142559


In [111]:
my_index = 0

**Use a cosine similarity measure to find users that are similar to us and have similar taste in books.**

**Cosine similarity will just find the similarity between two rows in our matrix so that we can find how similar each user is to us in terms of what books they read and how they rated them.**

In [112]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(ratings_mat[my_index,:], ratings_mat).flatten()

In [113]:
similarity[0]

1.0000000000000013

**We will find the indices of the users who are most similar to us - That is what this numpy r partition function does - We're passing in negative 15. So what we're going to find are the 15 users who have the most similar taste.**

In [114]:
import numpy as np

indices = np.argpartition(similarity, -15)[-15:]

In [115]:
indices

array([ 33,  89,  90,  29, 125,  45,  39,  78,  42,  44,  93,  98, 119,
       129,   0])

**Find all of the rows in interactions where the user index is in our indices.**

In [116]:
similar_users = interactions[interactions["user_index"].isin(indices)].copy()

**We'll just take ourselves out so we do not get book recommendations from ourselves.**

In [117]:
similar_users = similar_users[similar_users["user_id"]!="-1"]

In [118]:
similar_users

,user_id,book_id,rating,user_index,book_index
608084,180813,8492825,5,29,438391
608085,180813,914121,3,29,448046
608086,180813,4374400,4,29,365325
608087,180813,13638020,0,29,61292
608088,180813,12983100,5,29,41986
...,...,...,...,...,...
1087579,320956,25493853,0,129,244474
1087580,320956,35097545,0,129,347011
1087581,320956,30040068,0,129,304611
1087582,320956,28269171,0,129,284450


**6. Creating Book Recommendations**

**How many times each book appeared in these recommendations.**

In [119]:
book_recs = similar_users.groupby("book_id").rating.agg(['count', 'mean'])

In [120]:
book_recs

,count,mean
book_id,,
1,13,3.769231
10016013,1,0.000000
1002108,1,0.000000
100237,1,2.000000
10025305,5,1.200000
...,...,...
99756,1,0.000000
9975679,2,0.000000
998578,1,4.000000


**Adding book titles.**

In [121]:
books_titles = pd.read_json("books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)

**Merge our 2 datasets to get the book titles into our recommendations.**

In [122]:
book_recs = book_recs.merge(books_titles, how="inner", on="book_id")

In [123]:
book_recs

,book_id,count,mean,title,ratings,url,cover_image,mod_title
0,1,13,3.769231,Harry Potter and the Half-Blood Prince (Harry ...,1713866,https://www.goodreads.com/book/show/1.Harry_Po...,https://images.gr-assets.com/books/1361039191m...,harry potter and the halfblood prince harry po...
1,10016013,1,0.000000,Harry Potter and the Methods of Rationality,8691,https://www.goodreads.com/book/show/10016013-h...,https://images.gr-assets.com/books/1293582551m...,harry potter and the methods of rationality
2,1002108,1,0.000000,Grace Above All,43,https://www.goodreads.com/book/show/1002108.Gr...,https://images.gr-assets.com/books/1312009382m...,grace above all
3,100237,1,2.000000,Monkey: The Journey to the West,2997,https://www.goodreads.com/book/show/100237.Monkey,https://s.gr-assets.com/assets/nophoto/book/11...,monkey the journey to the west
4,10025305,5,1.200000,"Clockwork Prince (The Infernal Devices, #2)",318750,https://www.goodreads.com/book/show/10025305-c...,https://images.gr-assets.com/books/1460477747m...,clockwork prince the infernal devices 2
...,...,...,...,...,...,...,...,...
10527,99756,1,0.000000,Love Story,583,https://www.goodreads.com/book/show/99756.Love...,https://images.gr-assets.com/books/1365659626m...,love story
10528,9975679,2,0.000000,"Ashes (Ashes Trilogy, #1)",19788,https://www.goodreads.com/book/show/9975679-ashes,https://s.gr-assets.com/assets/nophoto/book/11...,ashes ashes trilogy 1
10529,998578,1,4.000000,"Batgirl, Vol. 5: Kicking Assassins",249,https://www.goodreads.com/book/show/998578.Bat...,https://s.gr-assets.com/assets/nophoto/book/11...,batgirl vol 5 kicking assassins
10530,99944,1,0.000000,The Bhagavad Gita,33855,https://www.goodreads.com/book/show/99944.The_...,https://images.gr-assets.com/books/1383059639m...,the bhagavad gita


**7. Ranking Our Book Recommendations**

**Figuring out which recommendations are relevant to us.**

**We'll first create an adjusted count which is a count but normalized for how many times the book appeared among people similar to us.**

**Ratings is the number of times the book was rated across all of goodreads.**

In [124]:
book_recs["adjusted_count"] = book_recs["count"] * (book_recs["count"] / book_recs["ratings"])

**Score: Indicating how much we might like a book.**

In [125]:
book_recs["score"] = book_recs["mean"] * book_recs["adjusted_count"]

**Take out any books we already have read.**

**1st we'll take out any book where the book id matches an id of a book we already read. The 2nd thing we'll do is we'll take out any books where the title matches a book we've already read.**

In [126]:
book_recs = book_recs[~book_recs["book_id"].isin(my_books["book_id"])]

In [127]:
my_books["mod_title"] = my_books["title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True).str.lower()

**Replace any sequences of spaces.**

In [128]:
my_books["mod_title"] = my_books["mod_title"].str.replace("\s+", " ", regex=True)

**Let's take out anything in our recommendations where the mod title fits into the books we already read.**

In [129]:
book_recs = book_recs[~book_recs["mod_title"].isin(my_books["mod_title"])]

**Remove anything that appeared less than twice or less in our recommendations.**

In [130]:
book_recs = book_recs[book_recs["count"]>2]

**We want to make sure that we only find books where our mean rating is greater than a certain amount.**

In [131]:
book_recs = book_recs[book_recs["mean"] >=4]

**Creating our top recommendations which we'll sort based on our score (This basically sorts the dataframe based on the score).**

In [132]:
top_recs = book_recs.sort_values("mean", ascending=False)

In [133]:
top_recs

,book_id,count,mean,title,ratings,url,cover_image,mod_title,adjusted_count,score
7325,28862528,4,4.750000,"Saga, Vol. 6 (Saga, #6)",20367,https://www.goodreads.com/book/show/28862528-s...,https://images.gr-assets.com/books/1486028997m...,saga vol 6 saga 6,0.000786,0.003732
782,13455782,5,4.600000,"Shatter Me (Shatter Me, #1)",14847,https://www.goodreads.com/book/show/13455782-s...,https://images.gr-assets.com/books/1338924054m...,shatter me shatter me 1,0.001684,0.007746
1305,16034235,4,4.500000,"Throne of Glass (Throne of Glass, #1)",17954,https://www.goodreads.com/book/show/16034235-t...,https://images.gr-assets.com/books/1419066050m...,throne of glass throne of glass 1,0.000891,0.004010
7449,29237211,4,4.500000,"Saga, Vol. 7 (Saga, #7)",12115,https://www.goodreads.com/book/show/29237211-s...,https://images.gr-assets.com/books/1486029003m...,saga vol 7 saga 7,0.001321,0.005943
2491,18293427,3,4.333333,The Storied Life of A.J. Fikry,120231,https://www.goodreads.com/book/show/18293427-t...,https://images.gr-assets.com/books/1404582137m...,the storied life of aj fikry,0.000075,0.000324
9356,5,12,4.333333,Harry Potter and the Prisoner of Azkaban (Harr...,1876252,https://www.goodreads.com/book/show/5.Harry_Po...,https://images.gr-assets.com/books/1499277281m...,harry potter and the prisoner of azkaban harry...,0.000077,0.000333
5299,24157427,3,4.333333,Don't Get Caught,1136,https://www.goodreads.com/book/show/24157427-d...,https://images.gr-assets.com/books/1445361821m...,dont get caught,0.007923,0.034331
9177,4137,3,4.333333,Me Talk Pretty One Day,502992,https://www.goodreads.com/book/show/4137.Me_Ta...,https://s.gr-assets.com/assets/nophoto/book/11...,me talk pretty one day,0.000018,0.000078
7252,28671039,3,4.333333,Holding Up the Universe,1870,https://www.goodreads.com/book/show/28671039-h...,https://images.gr-assets.com/books/1462468499m...,holding up the universe,0.004813,0.020856
1112,15745753,14,4.285714,Eleanor & Park,537595,https://www.goodreads.com/book/show/15745753-e...,https://images.gr-assets.com/books/1341952742m...,eleanor park,0.000365,0.001563


**8. Improve The Display Of The Books**

In [134]:
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)

top_recs.style.format({'url': make_clickable, 'cover_image': show_image})

,book_id,count,mean,title,ratings,url,cover_image,mod_title,adjusted_count,score
7325,28862528,4,4.750000,"Saga, Vol. 6 (Saga, #6)",20367,Goodreads,,saga vol 6 saga 6,0.000786,0.003732
782,13455782,5,4.600000,"Shatter Me (Shatter Me, #1)",14847,Goodreads,,shatter me shatter me 1,0.001684,0.007746
1305,16034235,4,4.500000,"Throne of Glass (Throne of Glass, #1)",17954,Goodreads,,throne of glass throne of glass 1,0.000891,0.004010
7449,29237211,4,4.500000,"Saga, Vol. 7 (Saga, #7)",12115,Goodreads,,saga vol 7 saga 7,0.001321,0.005943
2491,18293427,3,4.333333,The Storied Life of A.J. Fikry,120231,Goodreads,,the storied life of aj fikry,0.000075,0.000324
9356,5,12,4.333333,"Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)",1876252,Goodreads,,harry potter and the prisoner of azkaban harry potter 3,0.000077,0.000333
5299,24157427,3,4.333333,Don't Get Caught,1136,Goodreads,,dont get caught,0.007923,0.034331
9177,4137,3,4.333333,Me Talk Pretty One Day,502992,Goodreads,,me talk pretty one day,0.000018,0.000078
7252,28671039,3,4.333333,Holding Up the Universe,1870,Goodreads,,holding up the universe,0.004813,0.020856
1112,15745753,14,4.285714,Eleanor & Park,537595,Goodreads,,eleanor park,0.000365,0.001563


**Testing**

In [155]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# Step 1: Load the "liked_books" dataset
liked_books = pd.read_csv("liked_books.csv")

# Step 2: Apply the recommendation algorithm to generate recommendations
recommendations = liked_books["book_id"].tolist()

# Step 3: Compare recommended books with the "liked_books" dataset
ground_truth_books = liked_books["book_id"].tolist()

# Step 4: Calculate evaluation metrics if there are recommendations
if recommendations:
    precision = precision_score(ground_truth_books, recommendations, average="micro")
    recall = recall_score(ground_truth_books, recommendations, average="micro")
    f1 = f1_score(ground_truth_books, recommendations, average="micro")

    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-Score:", f1)
else:
    print("No recommendations generated.")

Precision: 1.0
Recall: 1.0
F1-Score: 1.0


**_______________K-Nearest Neighbor_____________________**

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.neighbors import NearestNeighbors

# Step 1: Reading in books we like
my_books = pd.read_csv("liked_books.csv", index_col=0)
my_books["book_id"] = my_books["book_id"].astype(str)

In [ ]:
# Step 2: Mapping CSV book IDs to book IDs
csv_book_mapping = {}
with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.strip().split(",")
        csv_book_mapping[csv_id] = book_id

# Set that contains all of the unique books that we have read
book_set = set(my_books["book_id"])

In [ ]:
# Step 3: Finding similar users using KNN
interactions_list = []
with open("goodreads_interactions.csv") as f:
    while True:
        line = f.readline()
        if not line:
            break

        user_id, csv_id, _, rating, _ = line.strip().split(",")
        book_id = csv_book_mapping.get(csv_id)

        if book_id in book_set:
            interactions_list.append([user_id, book_id, rating])

In [ ]:
# Step 4: Creating the user-book matrix
interactions = pd.DataFrame(interactions_list, columns=["user_id", "book_id", "rating"])
interactions["book_id"] = interactions["book_id"].astype(str)
interactions["user_id"] = interactions["user_id"].astype(str)
interactions["rating"] = pd.to_numeric(interactions["rating"])
interactions["user_index"] = interactions["user_id"].astype("category").cat.codes
interactions["book_index"] = interactions["book_id"].astype("category").cat.codes
ratings_mat_coo = coo_matrix((interactions["rating"], (interactions["user_index"], interactions["book_index"])))
ratings_mat = ratings_mat_coo.tocsr()

k = 15  # Number of similar users to find
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(ratings_mat)

my_index = 0
distances, indices = knn_model.kneighbors(ratings_mat[my_index], n_neighbors=k+1)
similar_user_indices = indices.squeeze()[1:]

In [ ]:
# Step 5: Filter and process similar users data
similar_users = interactions[interactions["user_index"].isin(similar_user_indices)].copy()
similar_users = similar_users[similar_users["user_id"] != "-1"]

In [ ]:
# Step 6: Creating book recommendations
book_recs = similar_users.groupby("book_id").rating.agg(['count', 'mean'])
book_recs.reset_index(inplace=True)

In [ ]:
print("Number of rows in book_recs:", len(book_recs))

Number of rows in book_recs: 5


In [ ]:
book_recs

,book_id,count,mean
0,17235026,1,0.000000
1,18460392,1,0.000000
2,20910157,2,0.000000
3,9361589,1,0.000000
4,9418327,15,4.133333


In [ ]:
# Step 7: Adding book titles
books_titles = pd.read_json("books_titles.json")
books_titles["book_id"] = books_titles["book_id"].astype(str)

In [ ]:
book_recs = book_recs.merge(books_titles, how="inner", on="book_id")

In [ ]:
# Step 8: Ranking the book recommendations
book_recs["adjusted_count"] = book_recs["count"] * (book_recs["count"] / book_recs["ratings"])

In [ ]:
book_recs

,book_id,count,mean,title,ratings,url,cover_image,mod_title,adjusted_count
0,17235026,1,0.000000,The Girl with All the Gifts,105967,https://www.goodreads.com/book/show/17235026-t...,https://images.gr-assets.com/books/1403033579m...,the girl with all the gifts,0.000009
1,18460392,1,0.000000,All the Bright Places,142495,https://www.goodreads.com/book/show/18460392-a...,https://images.gr-assets.com/books/1404331702m...,all the bright places,0.000007
2,20910157,2,0.000000,Yes Please,262157,https://www.goodreads.com/book/show/20910157-y...,https://images.gr-assets.com/books/1402815435m...,yes please,0.000015
3,9361589,1,0.000000,The Night Circus,441272,https://www.goodreads.com/book/show/9361589-th...,https://images.gr-assets.com/books/1387124618m...,the night circus,0.000002
4,9418327,15,4.133333,Bossypants,523393,https://www.goodreads.com/book/show/9418327-bo...,https://images.gr-assets.com/books/1481509554m...,bossypants,0.000430


In [ ]:
book_recs["score"] = book_recs["mean"] * book_recs["adjusted_count"]

In [ ]:
book_recs

,book_id,count,mean,title,ratings,url,cover_image,mod_title,adjusted_count,score
0,17235026,1,0.000000,The Girl with All the Gifts,105967,https://www.goodreads.com/book/show/17235026-t...,https://images.gr-assets.com/books/1403033579m...,the girl with all the gifts,0.000009,0.000000
1,18460392,1,0.000000,All the Bright Places,142495,https://www.goodreads.com/book/show/18460392-a...,https://images.gr-assets.com/books/1404331702m...,all the bright places,0.000007,0.000000
2,20910157,2,0.000000,Yes Please,262157,https://www.goodreads.com/book/show/20910157-y...,https://images.gr-assets.com/books/1402815435m...,yes please,0.000015,0.000000
3,9361589,1,0.000000,The Night Circus,441272,https://www.goodreads.com/book/show/9361589-th...,https://images.gr-assets.com/books/1387124618m...,the night circus,0.000002,0.000000
4,9418327,15,4.133333,Bossypants,523393,https://www.goodreads.com/book/show/9418327-bo...,https://images.gr-assets.com/books/1481509554m...,bossypants,0.000430,0.001777


In [ ]:
book_recs = book_recs[~book_recs["book_id"].isin(my_books["book_id"])]

In [ ]:
book_recs

,book_id,count,mean,title,ratings,url,cover_image,mod_title,adjusted_count,score


In [ ]:
my_books["mod_title"] = my_books["title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True).str.lower()

In [ ]:
# Replace any sequences of spaces
my_books["mod_title"] = my_books["mod_title"].str.replace("\s+", " ", regex=True)

In [ ]:
# Remove recommendations where the mod title matches the books we already read
book_recs = book_recs[~book_recs["mod_title"].isin(my_books["mod_title"])]

In [ ]:
# Remove recommendations that appeared less than twice in our recommendations
book_recs = book_recs[book_recs["count"] > 2]

In [ ]:
# Remove recommendations with mean rating below a certain threshold
book_recs = book_recs[book_recs["mean"] >= 4]

In [ ]:
# Sort the recommendations based on score
top_recs = book_recs.sort_values("mean", ascending=False)

In [ ]:
# Step 9: Improve the display of the books
def make_clickable(val):
    return '<a target="_blank" href="{}">Goodreads</a>'.format(val, val)

def show_image(val):
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)

top_recs.style.format({'url': make_clickable, 'cover_image': show_image})

,book_id,count,mean,title,ratings,url,cover_image,mod_title,adjusted_count,score
